### ダウンロードファイルの末尾の数字
1954: 175836

num = 175836 + int(year) - 1955
1956: 175837
1957: 175838
...
1973: 175854

num = 175836 + int(year) - 1954
1974: 175856
...
1988: 175870

num = 175905 + int(year) - 1989

https://www.pref.hiroshima.lg.jp/uploaded/attachment/175905.zip
https://www.pref.hiroshima.lg.jp/uploaded/attachment/175963.zip


In [116]:
def search_num(year):
    if year == 1954:
        return 175836
    if year >=1956 and year <=1973:
        num = 175836 + int(year) -1955
        return num
    if year >= 1974 and year <=1988:
        num = 175836 + int(year) - 1954
        return num
    if year >= 1989:
        num = 175905 + int(year) - 1989
        return num

pandasで読み込んでxls->xlsxにしなきゃいけない。
pandasで読み込んでしまうと結合セルが分離されてしまう

from openpyxl import load_workbook
import pandas as pd

def extract_table_from_excel(filepath, year):
    wb = load_workbook(filepath, data_only=True)
    ws = wb.active

    # 結合セルの値をすべてのセルにコピー
    for merged in ws.merged_cells.ranges:
        merged_val = ws[merged.coord.split(":")[0]].value
        for row in ws[merged.min_row:merged.max_row+1]:
            for cell in row[merged.min_col-1:merged.max_col]:
                cell.value = merged_val


In [5]:
# NOTE: search_num 関数がスクリプト内で定義されていないため、
# ここにダミーの定義を追加しています。実際の用途に合わせて実装してください。
def search_num(year):
    # 例: 年とURLのパス番号を紐付けるロジックを実装
    # 実際のURLの構造に合わせて変更してください
    if 1988 >= year and year >= 1954:
        showa = 29+year-1954
        return "s"+str(showa)+"z"
    if year >=1989:
        return "h0"+str(year-1988)+"z"

In [8]:
import glob
import os

for year in range (1956, 1957):
    extract_dir = "/work/a06/rmori/spillover/data/raw/"+search_num(year)
    csv_output_dir = "/work/a06/rmori/spillover/data/"+str(year)

    # ステップ2: 各ファイル処理
    xls_files = glob.glob(os.path.join(extract_dir, "*.xlsx"))

In [12]:
import requests
import zipfile
import os
import pandas as pd
import re
import glob
import shutil
import numpy as np
from openpyxl import load_workbook # openpyxlのインポート
from openpyxl.utils import get_column_letter # 必要に応じて、列の文字変換に使うことがある


for year in range (1957, 1989):
    extract_dir = "/work/a06/rmori/spillover/data/raw/"+search_num(year)
    csv_output_dir = "/work/a06/rmori/spillover/data/"+str(year)
    # 出力フォルダ作成
    os.makedirs(csv_output_dir, exist_ok=True)

    # ヘルパー関数群
    def is_table_like(row):
        str_row = row.astype(str)
        num_cols = (str_row.str.contains(r'\d')).sum()
        return num_cols >= 2 and row.count() >= 4

    def clean_cell(val):
        if pd.isna(val):
            return None
        val_str = str(val).strip()

        # 数値または文字列はそのまま残すが、注釈的な記号（例: *, †, #）は除去
        val_str = re.sub(r'[＊*†#※‡]', '', val_str)  # よく使われる注釈記号
        val_str = val_str.strip()
        
        return val_str if val_str else None
    
    # ステップ2: 各ファイル処理
    xls_files = glob.glob(os.path.join(extract_dir, "*.xlsx"))

    for file in xls_files:
        try:
            # --- OpenPyXL で結合セルを処理する部分 ---
            # ExcelファイルをOpenPyXLで読み込み
            # data_only=True は、セルに数式が入っている場合に計算結果の値を取得します
            wb = load_workbook(file, data_only=True)
            ws = wb.active # アクティブなシートを選択

            # 結合セルの値をすべての結合範囲にコピー
            # 結合セルを処理する前に、結合セル範囲のリストをコピーしておく
            # 処理中にリストが変更されるのを避けるため
            for merged_range in list(ws.merged_cells.ranges): # list()でコピー
                merged_val = ws[merged_range.coord.split(":")[0]].value # 結合セルの左上隅の値を取得
                
                # 結合セルを解除 (必須: 値を書き込むために結合を解除する必要がある)
                ws.unmerge_cells(str(merged_range))

                # 結合されていたすべてのセルに値を書き込む
                for row_idx in range(merged_range.min_row, merged_range.max_row + 1):
                    for col_idx in range(merged_range.min_col, merged_range.max_col + 1):
                        ws.cell(row=row_idx, column=col_idx, value=merged_val)
            
            # 変更を一時ファイルに保存し、そのファイルをPandasで読み込む
            # 元ファイルを直接上書きするリスクを避けるため、一時ファイルを使うのが安全です。
            temp_excel_path = file.replace('.xls', '_temp.xlsx') # .xls を .xlsx に変換
            wb.save(temp_excel_path)
            # ----------------------------------------------------

            # OpenPyXLで前処理したExcelファイルをPandasで読み込み
            # ヘッダーなし、文字列として読み込み
            df_raw = pd.read_excel(temp_excel_path, header=None, dtype=str)
            
            # 一時ファイルを削除
            os.remove(temp_excel_path)

            table_start = df_raw.apply(is_table_like, axis=1).idxmax()

            header_row = max(0, table_start - 5)
            #if df_raw.iloc[header_row].count() < 2:
                #header_row = table_start

            df_table = df_raw.iloc[header_row:]

            # クリーン処理
            df_cleaned = pd.DataFrame([
                [clean_cell(val) for val in row]
                for row in df_table.values
            ])
            
            # 空行（全セルNone）を削除
            df_cleaned = df_cleaned.dropna(how='all').replace('', pd.NA).dropna(how='all')
            df_cleaned.reset_index(drop=True, inplace=True)
            
            
            # 出力ファイル名
            base_name = os.path.splitext(os.path.basename(file))[0]
            output_path = os.path.join(csv_output_dir, f"{base_name}.csv")
            df_cleaned.to_csv(output_path, index=False, header=False)
            print(f"✅ 保存完了: {output_path}")

        except Exception as e:
            print(f"❌ 処理失敗: {file} - エラー: {e}")

✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-y12.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-w04.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-f01.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-b08.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-o10.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-aa03.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-n16.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-z04.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-k01.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-t12.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-q05.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-d09.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-l06.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-o08.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-b10.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-g07.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1956/tone-v02.csv
✅ 保存完了: /work/a06/rmori/spillo

In [13]:
pd.read_csv('/work/a06/rmori/spillover/data/1956/tone-a01.csv')

,1 昭和31年4月1日調整 市 町 村 便 覧,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38
0,NaN,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...,本表は各調査時の結果を昭和31年4月1日現在の行政区劃に組替えたものである。\n但し市町村分...
1,昭和31年4月\n1日現在,昭和25年\n10月1日,昭和25年\n10月1日,昭和30年\n10月1日,昭和30年\n10月1日,昭和30年\n10月1日,人 口\n （人）,人 口\n （人）,昭和30年\n12月20日,昭和29年\n8月1日,...,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）
2,昭和31年4月\n1日現在,昭和25年\n10月1日,昭和25年\n10月1日,昭和30年\n10月1日,昭和30年\n10月1日,昭和30年\n10月1日,人 口\n （人）,人 口\n （人）,昭和30年\n12月20日,昭和29年\n8月1日,...,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）,昭和29年7月1日 事 業 所 従 業 者 数 （人）
3,市 町 村 名,土地面積\n（平方粁）,土地面積\n（平方粁）,世 帯 数\n（世帯）,世 帯 数\n（世帯）,総 数,男,女,選 挙\n有権者数\n（人）,林野面積\n （町）,...,卸売及び\n小売業,卸売及び\n小売業,金融及び\n保険業,金融及び\n保険業,不動\n産業,不動\n産業,運輸通信\n公益事業,運輸通信\n公益事業,サービス業,サービス業
4,総数,NaN,8424.01,NaN,475146,2149044,1047007,1101815,1241637,NaN,...,NaN,115420,NaN,18358,NaN,816,NaN,36675,NaN,81739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,西城町,NaN,225.63,NaN,2222,11513,5722,5778,6437,NaN,...,NaN,178,NaN,57,NaN,1,NaN,4,NaN,174
200,東城町,NaN,297.09,NaN,4317,21713,10707,11006,12297,NaN,...,NaN,801,NaN,165,NaN,-,NaN,47,NaN,629
201,口和村,NaN,107.53,NaN,1172,6290,3180,3121,3519,NaN,...,NaN,133,NaN,15,NaN,-,NaN,23,NaN,152
202,高野町,NaN,153.42,NaN,991,5459,2740,2719,2983,NaN,...,NaN,134,NaN,25,NaN,20,NaN,9,NaN,123


In [ ]:
import requests
import zipfile
import os
import pandas as pd
import re
import glob
import shutil
import numpy as np
from openpyxl import load_workbook # openpyxlのインポート
from openpyxl.utils import get_column_letter

for year in range (1957, 1958):
    zip_url = "https://www.pref.hiroshima.lg.jp/uploaded/attachment/"+str(search_num(year))+".zip"
    zip_path = "/work/a06/rmori/spillover/data/"+str(year)+".zip"
    extract_dir = "/work/a06/rmori/spillover/data/hiroshima_data/"+str(year)
    csv_output_dir = "/work/a06/rmori/spillover/data/"+str(year)

    # ZIPファイルダウンロード
    response = requests.get(zip_url)
    with open(zip_path, "wb") as f:
        f.write(response.content)

    os.makedirs(extract_dir, exist_ok=True)

    # ZIPファイルを開いて展開
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for member in zip_ref.infolist():
            # 正しいファイル名を復元
            try:
                fixed_name = member.filename.encode('cp437').decode('cp932')
            except UnicodeDecodeError:
                fixed_name = member.filename  # ダメならそのまま使う

            # フォルダ構造を無視して、ベース名のみ取り出し
            filename = os.path.basename(fixed_name)
            if not filename:  # ディレクトリならスキップ
                continue

            # 展開先のファイルパス
            target_path = os.path.join(extract_dir, filename)

            with zip_ref.open(member) as source, open(target_path, "wb") as target:
                shutil.copyfileobj(source, target)

    # 出力フォルダ作成
    os.makedirs(csv_output_dir, exist_ok=True)

    # ヘルパー関数群
    def is_table_like(row):
        str_row = row.astype(str)
        num_cols = (str_row.str.contains(r'\d')).sum()
        return num_cols >= 2 and row.count() >= 4

    def clean_cell(val):
        if pd.isna(val):
            return None
        val_str = str(val).strip()

        # 数値または文字列はそのまま残すが、注釈的な記号（例: *, †, #）は除去
        val_str = re.sub(r'[＊*†#※‡]', '', val_str)  # よく使われる注釈記号
        val_str = val_str.strip()
        
        return val_str if val_str else None

    # ステップ2: 各ファイル処理
    xls_files = glob.glob(os.path.join(extract_dir, "*.xls"))

    for file in xls_files:
        try:
            df_raw = pd.read_excel(file, header=None, dtype=str)

            table_start = df_raw.apply(is_table_like, axis=1).idxmax()

            header_row = max(0, table_start - 5)
            #if df_raw.iloc[header_row].count() < 2:
                #header_row = table_start

            df_table = df_raw.iloc[header_row:]

            # クリーン処理
            df_cleaned = pd.DataFrame([
                [clean_cell(val) for val in row]
                for row in df_table.values
            ])
            
            # 空行（全セルNone）を削除
            df_cleaned = df_cleaned.dropna(how='all').replace('', pd.NA).dropna(how='all')
            df_cleaned.reset_index(drop=True, inplace=True)

            #列のずれを戻す
            def is_table_like2(row):
                str_row = row.astype(str)
                temp_str_row = str_row.replace('nan', '').replace('None', '')
                non_digit = temp_str_row.apply(lambda x: len(re.sub(r'\d', '', x))).sum()
                num_cols = (str_row.str.contains(r'\d')).sum()
                return non_digit <=10 and num_cols >= 3
            
            table_start2 = df_cleaned.apply(is_table_like2, axis=1).idxmax()
            for i in range(df_cleaned.shape[1]):
                ind = df_cleaned.index.get_loc(table_start2)
                if i == 0:
                    continue
                if pd.isna(df_cleaned.iat[ind-1, i]):
                    if pd.isna(df_cleaned.iloc[ind:, i-1]).all():
                            df_cleaned.iat[ind-1, i] = df_cleaned.iat[ind-1, i-1]
                            df_cleaned.iat[ind-1, i-1] = np.nan             

            # 出力ファイル名
            base_name = os.path.splitext(os.path.basename(file))[0]
            output_path = os.path.join(csv_output_dir, f"{base_name}.csv")
            df_cleaned.to_csv(output_path, index=False, header=False)
            print(f"✅ 保存完了: {output_path}")

        except Exception as e:
            print(f"❌ 処理失敗: {file} - エラー: {e}")


## ボツ
やはり、結合せるのすべてのセルに値を挿入するという方法しかないよう

In [117]:
import requests
import zipfile
import os
import pandas as pd
import re
import glob
import shutil
import numpy as np
from openpyxl import load_workbook # openpyxlのインポート
from openpyxl.utils import get_column_letter

for year in range (1957, 1958):
    zip_url = "https://www.pref.hiroshima.lg.jp/uploaded/attachment/"+str(search_num(year))+".zip"
    zip_path = "/work/a06/rmori/spillover/data/"+str(year)+".zip"
    extract_dir = "/work/a06/rmori/spillover/data/hiroshima_data/"+str(year)
    csv_output_dir = "/work/a06/rmori/spillover/data/"+str(year)

    # ZIPファイルダウンロード
    response = requests.get(zip_url)
    with open(zip_path, "wb") as f:
        f.write(response.content)

    os.makedirs(extract_dir, exist_ok=True)

    # ZIPファイルを開いて展開
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for member in zip_ref.infolist():
            # 正しいファイル名を復元
            try:
                fixed_name = member.filename.encode('cp437').decode('cp932')
            except UnicodeDecodeError:
                fixed_name = member.filename  # ダメならそのまま使う

            # フォルダ構造を無視して、ベース名のみ取り出し
            filename = os.path.basename(fixed_name)
            if not filename:  # ディレクトリならスキップ
                continue

            # 展開先のファイルパス
            target_path = os.path.join(extract_dir, filename)

            with zip_ref.open(member) as source, open(target_path, "wb") as target:
                shutil.copyfileobj(source, target)

    # 出力フォルダ作成
    os.makedirs(csv_output_dir, exist_ok=True)

    # ヘルパー関数群
    def is_table_like(row):
        str_row = row.astype(str)
        num_cols = (str_row.str.contains(r'\d')).sum()
        return num_cols >= 2 and row.count() >= 4

    def clean_cell(val):
        if pd.isna(val):
            return None
        val_str = str(val).strip()

        # 数値または文字列はそのまま残すが、注釈的な記号（例: *, †, #）は除去
        val_str = re.sub(r'[＊*†#※‡]', '', val_str)  # よく使われる注釈記号
        val_str = val_str.strip()
        
        return val_str if val_str else None

    # ステップ2: 各ファイル処理
    xls_files = glob.glob(os.path.join(extract_dir, "*.xls"))

    for file in xls_files:
        try:
            df_raw = pd.read_excel(file, header=None, dtype=str)

            table_start = df_raw.apply(is_table_like, axis=1).idxmax()

            header_row = max(0, table_start - 5)
            #if df_raw.iloc[header_row].count() < 2:
                #header_row = table_start

            df_table = df_raw.iloc[header_row:]

            # クリーン処理
            df_cleaned = pd.DataFrame([
                [clean_cell(val) for val in row]
                for row in df_table.values
            ])
            
            # 空行（全セルNone）を削除
            df_cleaned = df_cleaned.dropna(how='all').replace('', pd.NA).dropna(how='all')
            df_cleaned.reset_index(drop=True, inplace=True)

            #列のずれを戻す
            def is_table_like2(row):
                str_row = row.astype(str)
                temp_str_row = str_row.replace('nan', '').replace('None', '')
                non_digit = temp_str_row.apply(lambda x: len(re.sub(r'\d', '', x))).sum()
                num_cols = (str_row.str.contains(r'\d')).sum()
                return non_digit <=10 and num_cols >= 3
            
            table_start2 = df_cleaned.apply(is_table_like2, axis=1).idxmax()
            for i in range(df_cleaned.shape[1]):
                ind = df_cleaned.index.get_loc(table_start2)
                if i == 0:
                    continue
                if pd.isna(df_cleaned.iat[ind-1, i]):
                    if pd.isna(df_cleaned.iloc[ind:, i-1]).all():
                            df_cleaned.iat[ind-1, i] = df_cleaned.iat[ind-1, i-1]
                            df_cleaned.iat[ind-1, i-1] = np.nan             

            # 出力ファイル名
            base_name = os.path.splitext(os.path.basename(file))[0]
            output_path = os.path.join(csv_output_dir, f"{base_name}.csv")
            df_cleaned.to_csv(output_path, index=False, header=False)
            print(f"✅ 保存完了: {output_path}")

        except Exception as e:
            print(f"❌ 処理失敗: {file} - エラー: {e}")


✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-n04.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-m13.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-s08.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-t02.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-t05.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-n03.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-f10.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-m14.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-e07.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-b03.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-e09.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-s01.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-x05.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-s06.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-x02.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-b04.csv
✅ 保存完了: /work/a06/rmori/spillover/data/1957/tone-k06.csv
✅ 保存完了: /work/a06/rmori/spillov

In [ ]:
import requests
import zipfile
import os
import pandas as pd
import re
import glob
import shutil
import numpy as np
from openpyxl import load_workbook # openpyxlのインポート
from openpyxl.utils import get_column_letter

for year in range (1957, 1958):
    zip_url = "https://www.pref.hiroshima.lg.jp/uploaded/attachment/"+str(search_num(year))+".zip"
    zip_path = "/work/a06/rmori/spillover/data/"+str(year)+".zip"
    extract_dir = "/work/a06/rmori/spillover/data/hiroshima_data/"+str(year)
    csv_output_dir = "/work/a06/rmori/spillover/data/"+str(year)

    # ZIPファイルダウンロード
    response = requests.get(zip_url)
    with open(zip_path, "wb") as f:
        f.write(response.content)

    os.makedirs(extract_dir, exist_ok=True)

    # ZIPファイルを開いて展開
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for member in zip_ref.infolist():
            # 正しいファイル名を復元
            try:
                fixed_name = member.filename.encode('cp437').decode('cp932')
            except UnicodeDecodeError:
                fixed_name = member.filename  # ダメならそのまま使う

            # フォルダ構造を無視して、ベース名のみ取り出し
            filename = os.path.basename(fixed_name)
            if not filename:  # ディレクトリならスキップ
                continue

            # 展開先のファイルパス
            target_path = os.path.join(extract_dir, filename)

            with zip_ref.open(member) as source, open(target_path, "wb") as target:
                shutil.copyfileobj(source, target)

    # 出力フォルダ作成
    os.makedirs(csv_output_dir, exist_ok=True)

    # ヘルパー関数群
    def is_table_like(row):
        str_row = row.astype(str)
        num_cols = (str_row.str.contains(r'\d')).sum()
        return num_cols >= 2 and row.count() >= 4

    def clean_cell(val):
        if pd.isna(val):
            return None
        val_str = str(val).strip()

        # 数値または文字列はそのまま残すが、注釈的な記号（例: *, †, #）は除去
        val_str = re.sub(r'[＊*†#※‡]', '', val_str)  # よく使われる注釈記号
        val_str = val_str.strip()
        
        return val_str if val_str else None

    # ステップ2: 各ファイル処理
    xls_files = glob.glob(os.path.join(extract_dir, "*.xls"))

    for file in xls_files:
        try:
            df_raw = pd.read_excel(file, header=None, dtype=str)

            table_start = df_raw.apply(is_table_like, axis=1).idxmax()

            header_row = max(0, table_start - 5)
            #if df_raw.iloc[header_row].count() < 2:
                #header_row = table_start

            df_table = df_raw.iloc[header_row:]

            # クリーン処理
            df_cleaned = pd.DataFrame([
                [clean_cell(val) for val in row]
                for row in df_table.values
            ])
            
            # 空行（全セルNone）を削除
            df_cleaned = df_cleaned.dropna(how='all').replace('', pd.NA).dropna(how='all')
            df_cleaned.reset_index(drop=True, inplace=True)

            #列のずれを戻す
            def is_table_like2(row):
                str_row = row.astype(str)
                temp_str_row = str_row.replace('nan', '').replace('None', '')
                non_digit = temp_str_row.apply(lambda x: len(re.sub(r'\d', '', x))).sum()
                num_cols = (str_row.str.contains(r'\d')).sum()
                return non_digit <=10 and num_cols >= 3
            
            table_start2 = df_cleaned.apply(is_table_like2, axis=1).idxmax()
            for i in range(df_cleaned.shape[1]):
                ind = df_cleaned.index.get_loc(table_start2)
                if i == 0:
                    continue
                if pd.isna(df_cleaned.iat[ind-1, i]):
                    if pd.isna(df_cleaned.iloc[ind:, i-1]).all():
                            df_cleaned.iat[ind-1, i] = df_cleaned.iat[ind-1, i-1]
                            df_cleaned.iat[ind-1, i-1] = np.nan             

            # 出力ファイル名
            base_name = os.path.splitext(os.path.basename(file))[0]
            output_path = os.path.join(csv_output_dir, f"{base_name}.csv")
            df_cleaned.to_csv(output_path, index=False, header=False)
            print(f"✅ 保存完了: {output_path}")

        except Exception as e:
            print(f"❌ 処理失敗: {file} - エラー: {e}")
